In [1]:
!pip install datasets -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

from datasets import load_dataset
from sentence_transformers import SentenceTransformer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [2]:
MODEL_NAME = "freelawproject/modernbert-embed-base_finetune_8192"
MAX_TOKENS = 8192
MAX_SENTENCE = int(0.002 * MAX_TOKENS)

MODEL = SentenceTransformer(MODEL_NAME)
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/82.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/596M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [3]:
MAX_SENTENCE

16

In [4]:
def split_text_into_chunks(text, tokenizer=TOKENIZER, max_tokens=MAX_TOKENS, num_overlap_sentences=MAX_SENTENCE) -> list[str]:
    """Split text into chunks based on sentences, not exceeding max_tokens, with sentence overlap"""

    # Split the text to sentences & encode sentences with tokenizer
    sentences = sent_tokenize(text)
    encoded_sentences = [
        tokenizer.encode(sentence, add_special_tokens=False)
        for sentence in sentences
    ]
    lead_text = "search_document: "
    lead_tokens = tokenizer.encode(lead_text)
    lead_len = len(lead_tokens)
    chunks = []
    current_chunks: list[str] = []
    current_token_counts = len(lead_tokens)

    for sentence_tokens in encoded_sentences:
        sentence_len = len(sentence_tokens)
        # if the current sentence itself is above max_tokens
        if lead_len + sentence_len > max_tokens:
            # store the previous chunk
            if current_chunks:
                chunks.append(lead_text + " ".join(current_chunks))
            # truncate the sentence and store the truncated sentence as its own chunk
            truncated_sentence = tokenizer.decode(
                sentence_tokens[: (max_tokens - len(lead_tokens))]
            )
            chunks.append(lead_text + truncated_sentence)

            # start a new chunk with no overlap (because adding the current sentence will exceed the max_tokens)
            current_chunks = []
            current_token_counts = lead_len
            continue

        # if adding the new sentence will cause the chunk to exceed max_tokens
        if current_token_counts + sentence_len > max_tokens:
            overlap_sentences = current_chunks[
                -max(0, num_overlap_sentences) :
            ]
            # store the previous chunk
            if current_chunks:
                chunks.append(lead_text + " ".join(current_chunks))

            overlap_token_counts = tokenizer.encode(
                " ".join(overlap_sentences), add_special_tokens=False
            )
            # If the sentence with the overlap exceeds the limit, start a new chunk without overlap.
            if (
                lead_len + len(overlap_token_counts) + sentence_len
                > max_tokens
            ):
                current_chunks = [tokenizer.decode(sentence_tokens)]
                current_token_counts = lead_len + sentence_len
            else:
                current_chunks = overlap_sentences + [
                    tokenizer.decode(sentence_tokens)
                ]
                current_token_counts = (
                    lead_len + len(overlap_token_counts) + sentence_len
                )
            continue

        # if within max_tokens, continue to add the new sentence to the current chunk
        current_chunks.append(tokenizer.decode(sentence_tokens))
        current_token_counts += len(sentence_tokens)

    # store the last chunk if it has any content
    if current_chunks:
        chunks.append(lead_text + " ".join(current_chunks))
    return chunks

In [5]:
ds = load_dataset("freelawproject/opinions-metadata", split="train")
ds

README.md:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/503 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

Dataset({
    features: ['opinion_id', 'opinion', 'opinion_source', 'opinion_word_count', 'opinion_date_created', 'opinion_type', 'opinion_extracted_by_ocr', 'opinion_per_curiam', 'cluster_id', 'cluster_judges', 'cluster_nature_of_suit', 'cluster_source', 'cluster_blocked', 'cluster_precedential_status', 'cluster_citation_count', 'cluster_case_name', 'cluster_case_name_short', 'cluster_case_name_full', 'cluster_summary', 'cluster_history', 'cluster_headmatter', 'cluster_headnotes', 'cluster_posture', 'cluster_arguments', 'cluster_cross_reference', 'cluster_disposition', 'cluster_syllabus', 'docket_id', 'docket_number', 'docket_view_count', 'court_id', 'court_jurisdiction', 'court_in_use', 'court_short_name', 'court_full_name'],
    num_rows: 503
})

In [6]:
df = pd.DataFrame(ds)
df.head()

,opinion_id,opinion,opinion_source,opinion_word_count,opinion_date_created,opinion_type,opinion_extracted_by_ocr,opinion_per_curiam,cluster_id,cluster_judges,...,cluster_disposition,cluster_syllabus,docket_id,docket_number,docket_view_count,court_id,court_jurisdiction,court_in_use,court_short_name,court_full_name
0,4200405,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,opinion_plain_text,83,2017-08-31 15:10:30.817226+00,010combined,f,f,4423152,None,...,None,None,6146166,17-1492,0,fladistctapp,SA,t,District Court of Appeal of Florida,District Court of Appeal of Florida
1,2251008,889 N.E.2d 809 (2005) 356 Ill. App.3d 1130 PEO...,opinion_html_with_citations,23,2013-10-30 08:51:32.80475+00,010combined,f,f,2251008,None,...,None,None,2102724,1-03-2985,0,illappct,SA,t,Appellate Court of Illinois,Appellate Court of Illinois
2,4858364,"*234In re Cordes, Paul; Hurdlik, Paulette;—Pl...",opinion_xml_harvard,33,2021-08-26 01:28:25.294915+00,020lead,t,f,5044497,None,...,None,None,60294368,No. 2010-C-0584,0,la,S,t,Supreme Court of Louisiana,Supreme Court of Louisiana
3,2790710,Twersky v Incorporated Vil. of Great Neck ( 2...,opinion_html_with_citations,644,2015-04-01 19:05:17.073529+00,010combined,f,f,2790710,"Austin, Barros, Cohen, Eng",...,None,None,2646175,2014-07552,0,nyappdiv,SA,t,Appellate Division of the Supreme Court of New...,Appellate Division of the Supreme Court of the...
4,1470544,951 A.2d 180 (2008) Philip S. HORNER v. GOVERN...,opinion_html_with_citations,1472,2013-10-30 06:23:26.439857+00,010combined,f,f,1470544,"Broderick, Dalianis, Duggan, Galway, Hicks",...,None,None,1621241,2007-668,0,nh,S,t,Supreme Court of New Hampshire,Supreme Court of New Hampshire


In [7]:
df = df[["opinion_id", "opinion"]]

for index, row in df.iterrows():
  chunks = split_text_into_chunks(row["opinion"])
  embeddings = MODEL.encode(chunks)
  mean_embedding = np.mean(embeddings, axis=0)
  max_embedding = np.max(embeddings, axis=0)
  df.at[index, "mean_embedding"] = mean_embedding.astype(object)
  df.at[index, "max_embedding"] = max_embedding.astype(object)

df.head()

W0402 01:35:49.136000 499 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


,opinion_id,opinion,mean_embedding,max_embedding
0,4200405,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,"[0.04071538895368576, -0.01509640458971262, 0....","[0.04071538895368576, -0.01509640458971262, 0...."
1,2251008,889 N.E.2d 809 (2005) 356 Ill. App.3d 1130 PEO...,"[-0.006261525209993124, -0.016123848035931587,...","[-0.006261525209993124, -0.016123848035931587,..."
2,4858364,"*234In re Cordes, Paul; Hurdlik, Paulette;—Pl...","[-0.007931653410196304, 0.014965510927140713, ...","[-0.007931653410196304, 0.014965510927140713, ..."
3,2790710,Twersky v Incorporated Vil. of Great Neck ( 2...,"[-0.01694868877530098, 0.08422277867794037, 0....","[-0.01694868877530098, 0.08422277867794037, 0...."
4,1470544,951 A.2d 180 (2008) Philip S. HORNER v. GOVERN...,"[0.006950892973691225, 0.04788188636302948, 0....","[0.006950892973691225, 0.04788188636302948, 0...."


In [8]:
from google.colab import drive
drive.mount('/content/drive')

df["max_embedding"] = df["max_embedding"].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
df["mean_embedding"] = df["mean_embedding"].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

file_path = '/content/drive/MyDrive/similar_cases/8192_embeddings.json'
df.to_json(file_path, orient="records", indent=2)

Mounted at /content/drive
